#### K-Nearest Neighbors (KNN) Classification
---

K-nearest neighbors classification is (as its name implies) a classification model that uses the "K" most similar observations in order to make a prediction.

KNN is a supervised learning method; therefore, the training data must have known target values.

The process of of prediction using KNN is fairly straightforward:

1. Pick a value for K.
2. Search for the K observations in the data that are "nearest" to the measurements of the unknown iris.
    - Euclidian distance is often used as the distance metric, but other metrics are allowed.
3. Use the most popular response value from the K "nearest neighbors" as the predicted response value for the unknown iris.

The visualizations below show how a given area can change in its prediction as K changes.

- This is simulated data with two predictors
- Colored points represent true values and colored areas represent a **prediction space**. (This is called a Voronoi Diagram.)
- Each prediction space is wgere the majority of the "K" nearest points are the color of the space.
- To predict the class of a new point, we guess the class corresponding to the color of the space it lies in.

##### KNN Classification Map for Iris (K=1)

![1NN classification map](iris_01nn_map.png)

##### KNN Classification Map for Iris (K=5)

![5NN classification map](iris_05nn_map.png)

##### KNN Classification Map for Iris (K=15)

![15NN classification map](iris_15nn_map.png)

##### KNN Classification Map for Iris (K=50)

![50NN classification map](iris_50nn_map.png)

We can see that, as K increases, the classification spaces' borders become more distinct. However, you can also see that the spaces are not perfectly pure when it comes to the known elements within them.

**How are outliers affected by K?** As K increases, outliers are "smoothed out". Look at the above three plots and notice how outliers strongly affect the prediction space when K=1. When K=50, outliers no longer affect region boundaries. This is a classic bias-variance tradeoff -- with increasing K, the bias increases but the variance decreases.

<div style="color:blue;font-size:125%">
- What happens when K $\rightarrow$ number of points in the sample?
</div>
<div style="color:blue;font-size:125%">
- What is the best value for K?
</div>

##### NBA Position KNN Classifier

This dataset containing the 2015 season statistics for ~500 NBA players. The columns we'll use for features (and the target 'pos') are:

| Column | Meaning |
| ---    | ---     |
| pos | C: Center. F: Front. G: Guard |
| ast | Assists per game | 
| stl | Steals per game | 
| blk | Blocks per game |
| tov | Turnovers per game | 
| pf  | Personal fouls per game | 

For information about the other columns, see [this glossary](https://www.basketball-reference.com/about/glossary.html).

In [ ]:
# Read the NBA data into a DataFrame.
import pandas as pd

path = 'NBA_players_2015.csv'
nba = pd.read_csv(path, index_col=0)

In [ ]:
nba.head(5)

In [ ]:
# Map positions to numbers
nba['pos_num'] = nba.pos.map({'C':0, 'F':1, 'G':2})

In [ ]:
# Create feature matrix (X).
feature_cols = ['ast', 'stl', 'blk', 'tov', 'pf']
X = nba[feature_cols]

In [ ]:
X.head(5)

In [ ]:
X.describe()

In [ ]:
# Create response vector (y).
y = nba.pos_num
y.shape

<a id="using-the-traintest-split-procedure-k"></a>
### Using the Train/Test Split Procedure (K=1)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

#### Step 1: Split X and y into training and testing sets (using `random_state` for reproducibility).

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=99)

#### Step 2: Train the model on the training set (using K=1).

In [ ]:
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, y_train)

*  `algorithm=auto`, `leaf_size`  -- brute force or tree-based contruction algorithm;  does not affect solution; parameters to improve speed
*  `minkowski` is a family of distance metrics;  with `p=2` this is Euclidian distance, sum of squared distances
*  `weights` alternative to `uniform` is to weight closer points more heavily

#### Step 3: Test the model on the testing set and check the accuracy.

In [ ]:
y_pred_class = knn.predict(X_test)
print((metrics.accuracy_score(y_test, y_pred_class)))

In [ ]:
knn = KNeighborsClassifier(n_neighbors=50)
knn.fit(X, y)
y_pred_class = knn.predict(X)
print((metrics.accuracy_score(y, y_pred_class)))

<span style="color:blue; font-size:120%">Exercise</span>
* Train for `n_neighbors=1` on the entire dataset then test it on the entire dataset. What accuracy do you expect to get?  If you get something different, why?


#### Repeating for K=50

In [ ]:
knn = KNeighborsClassifier(n_neighbors=50)
knn.fit(X_train, y_train)
y_pred_class = knn.predict(X_test)
print((metrics.accuracy_score(y_test, y_pred_class)))

<span style="color:blue; font-size:120%">Exercise</span>
* Train and test on the entire data set, but using 50-KNN. Would we expect the accuracy to be higher, lower, or the same as compared to 1-KNN?

#### Comparing Testing Accuracy With Null Accuracy

Null accuracy is the accuracy that can be achieved by **always predicting the most frequent class**. For example, if most players are Centers, we would always predict Center.

The null accuracy is a benchmark against which you may want to measure every classification model.

#### Examine the class distribution from the training set.

Remember that we are comparing KNN to this simpler model. So, we must find the most frequent class **of the training set**.

In [ ]:
most_freq_class = y_test.value_counts().index[0]

print(y_test.value_counts())
print(most_freq_class)
print(150 / (150 + 140 + 68))

#### Compute null accuracy.

In [ ]:
y_test.value_counts()[most_freq_class] / len(y_test)

##### Tuning a KNN Model

Tuning means find an optimal value for K -- what does optimal mean?

In [ ]:
scores = []
for k in range(1,478):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X,y)
    pred = knn.predict(X)
    score = float(sum(pred == y)) / len(y)
    scores.append([k, score])

In [ ]:
data = pd.DataFrame(scores,columns=['k','score'])
data.plot.line(x='k',y='score');

**Question:** As K increases, why does the accuracy fall?

**Answer:** ...

#### Search for the "best" value of K.

In [ ]:
X_train.shape

In [ ]:
# Calculate TRAINING ACCURACY and TESTING ACCURACY for K=1 through 358.

k_range = list(range(1, 359))
training_accuracies = []
testing_accuracies = []

for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    
    # Calculate training accuracy
    y_pred_class = knn.predict(X_train)
    training_accuracy = metrics.accuracy_score(y_train, y_pred_class)
    training_accuracies.append(training_accuracy)
    
    # Calculate testing error.
    y_pred_class = knn.predict(X_test)
    testing_accuracy = metrics.accuracy_score(y_test, y_pred_class)
    testing_accuracies.append(testing_accuracy)

In [ ]:
testing_accuracies

In [ ]:
# Create a DataFrame of K, training error, and testing error.
column_dict = {'K': k_range, 'training accuracy':training_accuracies, 'testing accuracy':testing_accuracies}
df = pd.DataFrame(column_dict).set_index('K').sort_index(ascending=False)
df.head()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Plot the relationship between K (HIGH TO LOW) and TESTING ACCURACY.
df.plot(y='testing accuracy');
plt.xlabel('Value of K for KNN');
plt.ylabel('Accuracy');

In [ ]:
# Find the minimum testing error and the associated K value.
df.sort_values('testing accuracy', ascending=False).head()

In [ ]:
# Alternative method:
max(list(zip(testing_accuracies, k_range)))

<a id="training-error-versus-testing-error"></a>
### Training Accuracy Versus Testing Accuracy

In [ ]:
# Plot the relationship between K (HIGH TO LOW) and both TRAINING Accuracy and TESTING Accuracy.
df.plot();
plt.xlabel('Value of K for KNN');
plt.ylabel('Accuracy');

- Remember that model complexity is greatest at $K=1$ -- as $K$ gets larger the model tends toward "predict at the mode"

- **Training accuracy** increases as model complexity increases (lower value of K).
- **Testing accuracy** will tend to be low (overfitting, too much complexity) then increase, then decrease (too little complexity)

Evaluating the training and testing accuracy is important. For example:

- If the training accuracy is much higher than the test accuracy, then our model is likely overfitting. 
- If the test accuracy starts decreasing as we vary a hyperparameter (K), we may be overfitting.
- If either accuracy plateaus, our model is likely underfitting (not complex enough).

#### Making Predictions on Out-of-Sample Data

Given the statistics of a (truly) unknown NBA player, how do we predict his position?

In [ ]:
import numpy as np

# Instantiate the model with the best-known parameters.
knn = KNeighborsClassifier(n_neighbors=14)

# Re-train the model with X and y (not X_train and y_train). Why?
knn.fit(X, y)

# Make a prediction for an out-of-sample observation.
knn.predict(np.array([2, 1, 0, 1, 2]).reshape(1, -1))

In [ ]:
## Remember any classifier does estimation followed by choice.
##   Often useful to know that underlying probabilities predicted.
##     
knn.predict_proba(np.array([2, 1, 0, 1, 2]).reshape(1, -1))

What could we conclude?

- When using KNN on this data set with these features, the **best value for K** is likely to be around 14.
- Given the statistics of an **unknown player**, we estimate that we would be able to correctly predict his position about 74% of the time.

<a id="standardizing-features"></a>
## Standardizing Features
---

There is one major issue that applies to many machine learning models: They are sensitive to feature scale. 



> KNN in particular is sensitive to feature scale because it (by default) uses the Euclidean distance metric. To determine closeness, Euclidean distance sums the square difference along each axis. So, if one axis has large differences and another has small differences, the former axis will contribute much more to the distance than the latter axis.

This means that it matters whether our feature are centered around zero and have similar variance to each other.

<span style="color:blue;size:120%">Exercise</span>
* Our $X$ variables are
  * `ast` -- assists
  * `stl` -- steals
  * `blk` -- blocked shots
  * `tov` -- turnovers
  * `pf`  -- personal fouls
  
Suppose the data set had been coded as follows
  * `pf` was measured on a thousand fold scale;  1000, 2000, 3000, etc
  * `blk` was measured on a scale starting at -100 and increasing by 0.01;  0 => -100, 1 => -99.9

Intuitively, what will be the effect of making `pf` bigger?

Try this using rescaled values for `pf` and `blk` as above.  Transform the variables, re-train the model at $K=14$ and compare the predictions to those made by the original model.

---

In [ ]:
Xw = X.copy()
Yw = y.copy()
Xw["pf"] = Xw["pf"] * 1000
Xw["blk"] = -100 + Xw["blk"]* .01
knn2 = KNeighborsClassifier(n_neighbors=14)
knn2.fit(Xw, Yw)
# Calculate testing error.
y_pred_class_2 = knn2.predict(Xw)
y_pred_class = knn.predict(X)
agree = y_pred_class == y_pred_class_2
print(agree[0:20])
print(sum(agree)/len(agree))

Normalization:  Convert all attributes to have a mean of zero and variance of 1

<a id="use-standardscaler-to-standardize-our-data"></a>
### Use `StandardScaler` to Standardize our Data

StandardScaler standardizes our data by subtracting the mean from each feature and dividing by its standard deviation.

#### Separate feature matrix and response for scikit-learn.

In [ ]:
# Create feature matrix (X).
feature_cols = ['ast', 'stl', 'blk', 'tov', 'pf']

X = nba[feature_cols]
y = nba.pos_num  # Create response vector (y).

#### First Create the train/test split.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=99)

#### Instantiate and fit `StandardScaler`.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler(copy=True)
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [ ]:
print(X_train[:,4].mean())
print(X_train[:,4].std())

#### Fit a KNN model and look at the testing error.
Can you find a number of neighbors that improves our results from before?

In [ ]:
k_range = list(range(1, 348))
training_accuracies = []
testing_accuracies = []

for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    
    # Calculate training accuracy
    y_pred_class = knn.predict(X)
    training_accuracy = metrics.accuracy_score(y, y_pred_class)
    training_accuracies.append(training_accuracy)
    
    # Calculate testing error.
    y_pred_class = knn.predict(X_test)
    testing_accuracy = metrics.accuracy_score(y_test, y_pred_class)
    testing_accuracies.append(testing_accuracy)


In [ ]:
# Find the best K

In [ ]:
# Calculate testing error.
best_k = ??
knn = KNeighborsClassifier(n_neighbors=best_k)
knn.fit(X_train, y_train)

y_pred_class = knn.predict(X_test)
testing_accuracy = metrics.accuracy_score(y_test, y_pred_class)

print(testing_accuracy)

<a id="comparing-knn-with-other-models"></a>
## Comparing KNN With Other Models
---

**Advantages of KNN:**

- It's simple to understand and explain.
- Model training is fast.
- It can be used for classification and regression (for regression, take the average value of the K nearest points!).
- Being a non-parametric method, it is often successful in classification situations where the decision boundary is very irregular.

**Disadvantages of KNN:**

- It must store all of the training data.
- Its prediction phase can be slow when n is large.
- It is sensitive to irrelevant features.
- It is sensitive to the scale of the data.
- Accuracy is (generally) not competitive with the best supervised learning methods.